In [ ]:
pip install pandas openpyxl unidecode


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 3.7 MB/s eta 0:00:00


In [ ]:
import re
import pandas as pd
from unidecode import unidecode

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# =========================
# CONFIG - AJUSTE AQUI
# =========================
# Exemplo: "/content/drive/MyDrive/Rotary Analytics"

MUNICIPIOS_XLSX = "/content/drive/MyDrive/Rotary Analytics/municipios_distritos_v2.xlsx"
ROTARY_CLUBS_CSV = "/content/drive/MyDrive/Rotary Analytics/xxxxxxxxxx"


In [ ]:
OUT_DATASET_XLSX = "/content/drive/MyDrive/Rotary Analytics/rotary_clubs_with_municipio_pop_v4.xlsx"
OUT_EXCEPTIONS_XLSX = "/content/drive/MyDrive/Rotary Analytics/join_exceptions_missing_population_v4.xlsx"

In [ ]:
# =========================
# NORMALIZAÇÃO
# =========================
def norm_city(s: str) -> str:
    if pd.isna(s):
        return ""
    s = unidecode(str(s)).lower().strip()
    s = re.sub(r"[^\w\s]", " ", s)
    s = re.sub(r"\b(da|de|do|das|dos|d)\b", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def norm_uf(s: str) -> str:
    if pd.isna(s):
        return ""
    s = unidecode(str(s)).upper().strip()
    s = re.sub(r"[^A-Z]", "", s)
    m = re.search(r"(AC|AL|AP|AM|BA|CE|DF|ES|GO|MA|MT|MS|MG|PA|PB|PR|PE|PI|RJ|RN|RS|RO|RR|SC|SE|SP|TO)", s)
    return m.group(1) if m else ""

STATE_MAP = {
    "SAOPAULO":"SP","PARANA":"PR","RIOGRANDEDOSUL":"RS","SANTACATARINA":"SC",
    "MINASGERAIS":"MG","RIODEJANEIRO":"RJ","MATOGROSSO":"MT","MATOGROSSODOSUL":"MS",
    "ESPIRITOSANTO":"ES","BAHIA":"BA","CEARA":"CE","PIAUI":"PI","GOIAS":"GO",
    "DISTRITOFEDERAL":"DF","PERNAMBUCO":"PE","PARA":"PA","PARAIBA":"PB",
    "MARANHAO":"MA","AMAZONAS":"AM","RONDONIA":"RO","RORAIMA":"RR","AMAPA":"AP",
    "TOCANTINS":"TO","SERGIPE":"SE","ALAGOAS":"AL","ACRE":"AC","RIOGRANDEDONORTE":"RN"
}

def uf_from_field(raw: str) -> str:
    s = unidecode(str(raw)).upper()
    s = re.sub(r"[^A-Z]", "", s)
    if s in STATE_MAP:
        return STATE_MAP[s]
    if len(s) == 2:
        return s
    return norm_uf(s)

In [ ]:
# =========================
# LEITURA / LIMPEZA MUNICÍPIOS
# =========================
def load_municipios(path_xlsx: str) -> pd.DataFrame:
    mun = pd.read_excel(path_xlsx, engine="openpyxl")

    # Esperado: Distrito | NOME DO MUNICÍPIO | UF | codigo_ibge | POPULAÇÃO ESTIMADA
    mun = mun.rename(columns={
        "Distrito": "district",
        "NOME DO MUNICÍPIO": "city",
        "UF": "uf",
        "POPULAÇÃO ESTIMADA": "pop",
        "codigo_ibge": "codigo_ibge"
    })

    mun["district"] = pd.to_numeric(mun["district"], errors="coerce").astype("Int64")
    mun["uf"] = mun["uf"].apply(norm_uf)
    mun["city_norm"] = mun["city"].apply(norm_city)
    mun["pop"] = pd.to_numeric(mun["pop"], errors="coerce")
    mun["codigo_ibge"] = pd.to_numeric(mun["codigo_ibge"], errors="coerce")

    mun = mun.dropna(subset=["district", "city_norm"])
    mun = mun.drop_duplicates(["district","city_norm","uf","codigo_ibge","pop"])
    return mun


In [ ]:
# =========================
# LEITURA / LIMPEZA ROTARY
# =========================
def load_rotary(path_csv: str) -> pd.DataFrame:
    # Seu arquivo original do BI está com separador ";".
    # Encoding: cp1252 é o mais comum em export do Windows/Excel.
    rot = pd.read_csv(path_csv, sep=";", encoding="cp1252")

    rot["district"] = pd.to_numeric(rot["District"], errors="coerce").astype("Int64")
    rot["city_raw"] = rot["City"]
    rot["city_norm"] = rot["City"].apply(norm_city)
    rot["uf_best"] = rot["State/Province"].apply(uf_from_field)
    return rot

In [ ]:
# =========================
# PREFIX MATCH por distrito
# =========================
def build_candidates_by_district(mun: pd.DataFrame):
    return mun.groupby("district")["city_norm"].apply(lambda s: set(s)).to_dict()

def map_to_municipality(district, city_norm, cand_by_district):
    if not city_norm or pd.isna(city_norm) or pd.isna(district):
        return ""
    cand = cand_by_district.get(int(district), set())
    if not cand:
        return ""
    if city_norm in cand:
        return city_norm

    toks = city_norm.split()
    for k in range(len(toks), 0, -1):
        pref = " ".join(toks[:k])
        if pref in cand:
            return pref
    return ""


In [ ]:
# =========================
# JOIN V2
# =========================
def join_v2(rot: pd.DataFrame, mun: pd.DataFrame) -> pd.DataFrame:
    cand_by_district = build_candidates_by_district(mun)

    rot = rot.copy()
    rot["city_norm2"] = rot.apply(
        lambda r: (map_to_municipality(r["district"], r["city_norm"], cand_by_district) or r["city_norm"]),
        axis=1
    )

    mun_key = mun[["district","city_norm","uf","pop","codigo_ibge","city"]].drop_duplicates()
    mun_key = mun_key.rename(columns={"city_norm": "city_norm2"})

    # 1) merge forte
    m1 = rot.merge(
        mun_key,
        left_on=["district","city_norm2","uf_best"],
        right_on=["district","city_norm2","uf"],
        how="left",
        suffixes=("","_mun")
    )

    # 2) fallback: só quando city+district for único UF na base de municípios
    mun_counts = mun.groupby(["district","city_norm"]).size().reset_index(name="n_uf")
    mun_unique = mun_counts[mun_counts["n_uf"] == 1][["district","city_norm"]]
    mun_unique_table = mun.merge(mun_unique, on=["district","city_norm"], how="inner")[
        ["district","city_norm","uf","pop","codigo_ibge","city"]
    ].drop_duplicates().rename(columns={"city_norm":"city_norm2"})

    need2 = m1["pop"].isna()
    tmp = m1.loc[need2, ["district","city_norm2"]].copy()
    tmp2 = tmp.merge(mun_unique_table, on=["district","city_norm2"], how="left")

    m1.loc[need2, "uf"] = tmp2["uf"].values
    m1.loc[need2, "pop"] = tmp2["pop"].values
    m1.loc[need2, "codigo_ibge"] = tmp2["codigo_ibge"].values
    m1.loc[need2, "city"] = tmp2["city"].values

    return m1


def build_exceptions(final_df: pd.DataFrame) -> pd.DataFrame:
    exc = final_df[final_df["pop"].isna()].copy()
    out = (exc.groupby(["district","city_norm2"], as_index=False)
           .agg(n_clubs=("Club ID","nunique"),
                members=("Current Member Count","sum"),
                example_city=("city_raw", lambda x: x.value_counts().index[0] if x.value_counts().size else "")))
    out = out.sort_values(["n_clubs","members"], ascending=False)
    return out


In [ ]:
# =========================
# RUN
# =========================
mun = load_municipios(MUNICIPIOS_XLSX)
rot = load_rotary(ROTARY_CLUBS_CSV)

final = join_v2(rot, mun)

match_rate = final["pop"].notna().mean()
print(f"MATCH RATE (pop preenchida): {match_rate:.2%}")
print(f"Total linhas (clubs): {len(final):,}")

exc = build_exceptions(final)
print(f"Exceções (sem pop): {len(exc):,}")

with pd.ExcelWriter(OUT_DATASET_XLSX, engine="openpyxl") as w:
    final.to_excel(w, index=False, sheet_name="dataset")

with pd.ExcelWriter(OUT_EXCEPTIONS_XLSX, engine="openpyxl") as w:
    exc.to_excel(w, index=False, sheet_name="exceptions")

print("\nArquivos gerados no Drive:")
print(" -", OUT_DATASET_XLSX)
print(" -", OUT_EXCEPTIONS_XLSX)

MATCH RATE (pop preenchida): 95.32%
Total linhas (clubs): 2,499
Exceções (sem pop): 81

Arquivos gerados no Drive:
 - /content/drive/MyDrive/Rotary Analytics/rotary_clubs_with_municipio_pop_v4.xlsx
 - /content/drive/MyDrive/Rotary Analytics/join_exceptions_missing_population_v4.xlsx


In [ ]:
final = join_v2(rot, mun)

In [ ]:
import numpy as np
import re

def is_state_abbr_like(x: str) -> bool:
    if x is None:
        return False
    s = str(x).strip().lower()
    # exemplos: "s p", "pr", "m g", "sp", "pr."
    s = re.sub(r"[^a-z]", "", s)
    return s in {"sp","pr","mg","rj","rs","sc","es","mt","ms","go","df","ba","ce","pe","pa","pb","ma","pi","rn","ro","rr","ap","am","ac","se","to"}

final["match_flag"] = np.where(final["pop"].notna(), "MATCHED", "UNMATCHED")

final["unmatched_reason"] = np.where(final["match_flag"] == "MATCHED", "",
    np.where(final["city_norm2"].isna() | (final["city_norm2"].astype(str).str.strip() == ""), "MISSING_CITY",
    np.where(final["city_norm2"].astype(str).str.len() <= 3, "SHORT_CITY_TOKEN",
    np.where(final["city_norm2"].apply(is_state_abbr_like), "STATE_ABBR_IN_CITY", "OTHER")
)))

# opcional: percentual matched já no dataset
final["match_rate_overall"] = final["match_flag"].eq("MATCHED").mean()


In [ ]:
exc = build_exceptions(final)

with pd.ExcelWriter(OUT_DATASET_XLSX, engine="openpyxl") as w:
    final.to_excel(w, index=False, sheet_name="dataset")

with pd.ExcelWriter(OUT_EXCEPTIONS_XLSX, engine="openpyxl") as w:
    exc.to_excel(w, index=False, sheet_name="exceptions")

In [ ]:
# =========================
# GERAR RELATÓRIO MENSAL (formato Diretor)
# =========================
import numpy as np
import pandas as pd

In [ ]:
# --- 1) Garantir colunas que vamos usar ---
# (ajusta automaticamente caso você tenha Club ID ou não)
club_id_col = "Club ID" if "Club ID" in final.columns else ("Club Name" if "Club Name" in final.columns else None)
if club_id_col is None:
    raise ValueError("Não encontrei 'Club ID' nem 'Club Name' no dataframe final.")

In [ ]:
# --- 2) Agregar Rotary por municipio/distrito (a partir do final) ---
agg_rotary = (
    final
    .groupby(["district", "city_norm2"], as_index=False)
    .agg(
        ASSOCIADOS=("Current Member Count", "sum"),
        QTDE_CLUBES=(club_id_col, "nunique"),
    )
)

In [ ]:
# --- 3) Base municipal (garantir chave equivalente) ---
# mun tem: district, city_norm, uf, pop, codigo_ibge, city
mun_base = mun.copy()
mun_base["city_norm2"] = mun_base["city_norm"]  # chave para casar com o final

In [ ]:
# --- 4) Left join: TODOS os municípios entram (inclusive sem clube) ---
rel = mun_base.merge(
    agg_rotary,
    on=["district", "city_norm2"],
    how="left"
)

In [ ]:
# preencher zeros onde não tinha clube
rel["ASSOCIADOS"] = rel["ASSOCIADOS"].fillna(0).astype(int)
rel["QTDE_CLUBES"] = rel["QTDE_CLUBES"].fillna(0).astype(int)

In [ ]:
# --- 5) Habitantes por associado (per capta do diretor) ---
# Se ASSOCIADOS = 0, deixa em branco (ou pode deixar 0 / np.inf — eu recomendo vazio)
rel["HABITANTES_POR_ASSOCIADO"] = np.where(
    rel["ASSOCIADOS"] > 0,
    (rel["pop"] / rel["ASSOCIADOS"]).round(2),
    np.nan
)


In [ ]:
# --- 6) Montar colunas exatamente como o diretor pediu ---
relatorio_mensal = rel.rename(columns={
    "uf": "UF",
    "codigo_ibge": "COD_MUNICIPIO_IBGE",
    "city": "MUNICIPIO",
    "pop": "POPULACAO_ESTIMADA",
    "district": "DISTRITO"
})[
    [
        "UF",
        "COD_MUNICIPIO_IBGE",
        "MUNICIPIO",
        "POPULACAO_ESTIMADA",
        "DISTRITO",
        "ASSOCIADOS",
        "QTDE_CLUBES",
        "HABITANTES_POR_ASSOCIADO"
    ]
].sort_values(["UF", "DISTRITO", "MUNICIPIO"])

In [ ]:
# --- 7) Exportar Excel no Drive (e opcionalmente baixar) ---
# Sugestão: nome com mês para histórico
from datetime import datetime
ref_mes = datetime.now().strftime("%Y-%m")

OUT_RELATORIO_XLSX = "/content/drive/MyDrive/Rotary Analytics/Relatorio_Mensal_Rotary_Municipios_dezembro25.xlsx"

with pd.ExcelWriter(OUT_RELATORIO_XLSX, engine="openpyxl") as w:
    relatorio_mensal.to_excel(w, index=False, sheet_name="Relatorio_Mensal")
    exc.to_excel(w, index=False, sheet_name="Exceptions")

print("Relatório mensal gerado em dezembro")
print(" -", OUT_RELATORIO_XLSX)

Relatório mensal gerado em dezembro
 - /content/drive/MyDrive/Rotary Analytics/Relatorio_Mensal_Rotary_Municipios_dezembro25.xlsx
